In [3]:
pip uninstall numpy pandas -y

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.3.3
Uninstalling pandas-2.3.3:
  Successfully uninstalled pandas-2.3.3
Note: you may need to restart the kernel to use updated packages.


The value specified in an AutoRun registry key could not be parsed.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.


In [4]:
pip install numpy==2.0.2 pandas==2.3.3 --no-cache-dir --force-reinstall

The value specified in an AutoRun registry key could not be parsed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
gcs-oauth2-boto-plugin 3.0 requires rsa==4.7.2, but you have rsa 4.9 which is incompatible.
langchain 0.0.340 requires numpy<2,>=1, but you have numpy 2.0.2 which is incompatible.
matplotlib 3.8.2 requires numpy<2,>=1.21, but you have numpy 2.0.2 which is incompatible.
pandasai 1.5.4 requires pandas==1.5.3, but you have pandas 2.3.3 which is incompatible.
streamlit 1.28.2 requires numpy<2,>=1.19.3, but you have numpy 2.0.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.9 MB 2.6 MB/s eta 0:00:06
   ---------------------------------------- 0.2/15.9 MB 1.9 MB/s eta 0:00:09
    --------------------------------------- 0.2/15.9 MB 1.6 MB/s eta 0:00:11
    --------------------------------------- 0.3/15.9 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.5/15.9 MB 2.3 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/15.9 MB 2.8 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/15.9 MB 3.3 MB/s eta 0:00:05
   --- ------------------------------------ 1.4/15.9 MB 3.6 MB/s eta 0:00:05
   ---- ----------------------------------- 1.7/15.9 MB 4.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.9/15.9 MB 4.1 MB/s eta 0:00:04
   ----- -----

In [5]:
import pandas as pd

In [6]:
def check_timestamps(df, ts_col, utc = True):
    original = len(df)
    df[ts_col] = pd.to_datetime(df[ts_col], errors = 'coerce', utc = utc)
    invalid = df[ts_col].isna().sum()
    if invalid:
        print(f"Invalid/malformed timestamps: {invalid} rows → dropped")
        print(df[df[ts_col].isna()][ts_col].head(5))
        # log_anomaly("Invalid/malformed timestamps", invalid, "dropped", df[df[ts_col].isna()][ts_col].head(1).to_dict())
    df = df.dropna(subset = [ts_col])
    print(f"Timestamps parsed", len(df), "are valid")
    return df

In [7]:
def check_clock_drift(df, ts_col, vehicle_col = 'vehicle_id'):
    df = df.sort_values([vehicle_col, ts_col])
    df['prev_ts'] = df.groupby(vehicle_col)[ts_col].shift(1)
    df['time_diff'] = (df[ts_col] - df['prev_ts']).dt.total_seconds()
    backward = df[df['time_diff'] < 0]
    huge_gaps = df[df['time_diff'] > 86400]  # >1 day
    if not backward.empty:
        print("Clock jumped backward", len(backward), "flagged, kept (possible reset)")
        print(backward[[vehicle_col, ts_col, 'time_diff']].head())
    if not huge_gaps.empty:
        print("Huge time gaps (>24h)", len(huge_gaps), "flagged, kept")
        print(huge_gaps[[vehicle_col, ts_col, 'time_diff']].head())
    df = df.drop(columns = ['prev_ts', 'time_diff'], errors = 'ignore')
    return df

In [8]:
tlm_data = pd.read_csv('dataset/telemetry_data.csv')

In [9]:
tlm_data.drop(columns = ['Unnamed: 0'], inplace = True)
tlm_data.head()

,ID,VEHICLE_ID,TIMESTAMP,SPEED,IGNITION_STATUS,EV_BATTERY_LEVEL,ODOMETER
0,428bdd3a-243a-4735-b3be-4e12ec511774,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:45:42.018,NaN,NaN,NaN,NaN
1,b76e490b-8292-4b20-927c-5842bda6104a,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:58:08.017,NaN,NaN,NaN,NaN
2,40aabc8e-5a83-4772-bbd6-1d2634f20aa4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:57:02.031,NaN,NaN,NaN,38685.9
3,8f73a942-e549-4662-9f7e-e21f3d104637,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:38:57.002,0.0,on,NaN,NaN
4,852caadc-e582-4cd0-97cb-7f9a20c9634f,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 02:00:05.004,NaN,NaN,NaN,NaN


In [10]:
tlm_data = check_timestamps(tlm_data, 'TIMESTAMP')

Invalid/malformed timestamps: 93 rows → dropped
1073966   NaT
1075889   NaT
1079957   NaT
1090384   NaT
1097502   NaT
Name: TIMESTAMP, dtype: datetime64[ns, UTC]
Timestamps parsed 1833459 are valid


In [11]:
tlm_data = check_clock_drift(tlm_data, 'TIMESTAMP', 'VEHICLE_ID')

Huge time gaps (>24h) 87 flagged, kept
                                   VEHICLE_ID  \
1421073  04105a12-59b9-447b-865f-599f48eed1d7   
1084534  04105a12-59b9-447b-865f-599f48eed1d7   
1806270  04105a12-59b9-447b-865f-599f48eed1d7   
1157258  04105a12-59b9-447b-865f-599f48eed1d7   
1446087  04105a12-59b9-447b-865f-599f48eed1d7   

                               TIMESTAMP   time_diff  
1421073 2023-01-05 12:15:03.002000+00:00  241550.001  
1084534 2023-01-06 18:57:44.001000+00:00  110560.999  
1806270 2023-01-09 17:09:14.001000+00:00   86401.000  
1157258 2023-01-11 17:09:15.001000+00:00   86403.000  
1446087 2023-01-17 17:09:14.001000+00:00   86401.000  


In [12]:
trg_data = pd.read_csv('dataset/triggers_soc.csv')

In [13]:
trg_data.drop(columns = ['Unnamed: 0'], inplace = True)
trg_data.head()

,CTS,PNID,NAME,VAL
0,2021-12-01 17:58:27 IST+0530,206958332,IGN_CYL,OFF
1,2021-12-01 17:58:27 IST+0530,206958332,IGN_CYL,OFF
2,2021-12-01 17:56:24 IST+0530,206958332,IGN_CYL,ON
3,2021-12-01 17:56:24 IST+0530,206958332,CHARGE_STATE,50.1
4,2021-12-01 17:56:24 IST+0530,206958332,IGN_CYL,ON


In [14]:
print(trg_data.shape[0])
trg_data = trg_data.drop_duplicates()
print(trg_data.shape[0])

68670
59043


In [15]:
trg_data = check_timestamps(trg_data, 'CTS', utc = False)

Timestamps parsed 59043 are valid


In [16]:
trg_data = check_clock_drift(trg_data, 'CTS', 'PNID')

Huge time gaps (>24h) 201 flagged, kept
            PNID                       CTS   time_diff
38382  186776160 2021-09-09 05:36:21+05:30    343462.0
38851  186776160 2021-09-10 08:51:05+05:30     96958.0
40910  186776160 2021-09-16 05:26:06+05:30    444635.0
41419  186776160 2021-09-17 20:36:03+05:30    140027.0
50910  186776160 2022-01-12 23:30:43+05:30  10116790.0


In [17]:
map_data = pd.read_csv('dataset/vehicle_pnid_mapping.csv')

In [18]:
map_data.drop(columns = ['Unnamed: 0'], inplace = True)
map_data.head()

,ID,IDS
0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,"[\n ""256932585"",\n ""256932577"",\n ""25693258..."
1,ae912623-e122-4bfb-b2a1-7d407ea42b1b,NaN
2,39424065-b3be-45e5-8f7d-b7f16a6c190a,NaN
3,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,"[\n ""270199389"",\n ""270199393"",\n ""27019938..."
4,9893c80d-f282-46cf-a794-280f0306c936,"[\n ""256933093"",\n ""256933081"",\n ""25693308..."


In [19]:
map_data.rename(columns = {'IDS': 'PNID'}, inplace = True)

In [20]:
map_data['PNID'] = map_data['PNID'].apply(lambda x: eval(x) if x == x else [])

In [21]:
map_data = map_data.explode('PNID')

In [22]:
import json

In [23]:
with open('dataset/artificial_ign_off_data.json') as f:
    txt = json.load(f)

In [24]:
if not txt:
    ign_off_data =  pd.DataFrame(columns = ['VEHICLE_ID', 'event_ts', 'event'])

ign_off_data = pd.DataFrame(txt)

ign_off_data = ign_off_data.rename(columns = {'vehicleId': 'VEHICLE_ID', 'timestamp': 'event_ts'})
ign_off_data = ign_off_data[ign_off_data['type'] == 'artificial_event']  # safety
ign_off_data['event'] = 'ignitionOff'

In [25]:
ign_off_data = check_timestamps(ign_off_data, 'event_ts')

Timestamps parsed 411 are valid


In [26]:
ign_off_data = check_clock_drift(ign_off_data, 'event_ts', 'VEHICLE_ID')

Huge time gaps (>24h) 119 flagged, kept
                               VEHICLE_ID                         event_ts  \
380  19c4f06a-d5c7-4e51-ae2c-56c5367c0d50 2021-12-21 22:52:04.009000+00:00   
353  19c4f06a-d5c7-4e51-ae2c-56c5367c0d50 2021-12-24 00:18:41.048000+00:00   
341  19c4f06a-d5c7-4e51-ae2c-56c5367c0d50 2021-12-25 18:38:17.030000+00:00   
302  19c4f06a-d5c7-4e51-ae2c-56c5367c0d50 2021-12-30 17:43:07.024000+00:00   
279  19c4f06a-d5c7-4e51-ae2c-56c5367c0d50 2022-01-02 21:00:58.002000+00:00   

      time_diff  
380  104815.987  
353  110574.040  
341  152375.982  
302  341147.014  
279  258036.970  


In [27]:
tlm_data['ign_state'] = (tlm_data['IGNITION_STATUS'] == 'on').astype(int)
tlm_data = tlm_data.sort_values(['VEHICLE_ID', 'TIMESTAMP'])

In [28]:
tlm_data['prev_state'] = tlm_data.groupby('VEHICLE_ID')['ign_state'].shift(1)

In [29]:
tlm_events = tlm_data[(tlm_data['prev_state'] != tlm_data['ign_state']) & tlm_data['prev_state'].notna()].copy()

In [30]:
tlm_events['event'] = tlm_events['ign_state'].map({1: 'ignitionOn', 0: 'ignitionOff'})
tlm_events = tlm_events[['VEHICLE_ID', 'TIMESTAMP', 'event']].rename(columns = {'TIMESTAMP': 'event_ts'})

In [31]:
trg_data

,CTS,PNID,NAME,VAL
52902,2021-09-04 05:31:16+05:30,186776160,IGN_CYL,ON
52903,2021-09-04 05:31:18+05:30,186776160,CHARGE_STATE,89.0
52884,2021-09-04 05:45:40+05:30,186776160,IGN_CYL,OFF
52885,2021-09-04 05:45:43+05:30,186776160,CHARGE_STATE,89.0
52886,2021-09-04 05:53:16+05:30,186776160,IGN_CYL,ON
...,...,...,...,...
21086,2022-01-31 00:50:46+05:30,270768911,CHARGE_STATE,54.1
21095,2022-01-31 01:13:07+05:30,270768911,IGN_CYL,ON
21096,2022-01-31 01:13:09+05:30,270768911,CHARGE_STATE,54.1
21133,2022-01-31 01:23:47+05:30,270768911,IGN_CYL,OFF


In [32]:
trg_data = check_timestamps(trg_data, 'CTS', utc = False)

Timestamps parsed 59043 are valid


In [33]:
trg_data

,CTS,PNID,NAME,VAL
52902,2021-09-04 05:31:16+05:30,186776160,IGN_CYL,ON
52903,2021-09-04 05:31:18+05:30,186776160,CHARGE_STATE,89.0
52884,2021-09-04 05:45:40+05:30,186776160,IGN_CYL,OFF
52885,2021-09-04 05:45:43+05:30,186776160,CHARGE_STATE,89.0
52886,2021-09-04 05:53:16+05:30,186776160,IGN_CYL,ON
...,...,...,...,...
21086,2022-01-31 00:50:46+05:30,270768911,CHARGE_STATE,54.1
21095,2022-01-31 01:13:07+05:30,270768911,IGN_CYL,ON
21096,2022-01-31 01:13:09+05:30,270768911,CHARGE_STATE,54.1
21133,2022-01-31 01:23:47+05:30,270768911,IGN_CYL,OFF


In [34]:
trg_data['NAME'].unique()

array(['IGN_CYL', 'CHARGE_STATE', 'EV_CHARGE_STATE'], dtype=object)

In [35]:
trg_data.dtypes

CTS     datetime64[ns, UTC+05:30]
PNID                        int64
NAME                       object
VAL                        object
dtype: object

In [36]:
map_data['PNID'] = map_data['PNID'].apply(lambda x: int(x) if x == x else x)

In [37]:
trg_events = trg_data[trg_data['NAME'] == 'IGN_CYL'].copy()
trg_events['event'] = trg_events['VAL'].map({'ON': 'ignitionOn', 'OFF': 'ignitionOff'})
trg_events = trg_events[['PNID', 'CTS', 'event']].rename(columns = {'CTS': 'event_ts'})
trg_events = trg_events.merge(map_data, on = 'PNID')
trg_events = trg_events[['ID', 'event_ts', 'event']]

In [38]:
trg_events

,ID,event_ts,event
0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 07:53:54+05:30,ignitionOn
1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 07:54:28+05:30,ignitionOff
2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-02 01:39:00+05:30,ignitionOn
3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-02 01:42:34+05:30,ignitionOff
4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-02 02:35:03+05:30,ignitionOn
...,...,...,...
9930,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-31 04:32:03+05:30,ignitionOff
9931,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-31 04:42:35+05:30,ignitionOn
9932,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-31 04:42:35+05:30,ignitionOn
9933,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-31 04:51:53+05:30,ignitionOff


In [39]:
trg_events = trg_events.rename(columns = {'ID': 'VEHICLE_ID'})
trg_events['event_ts'] = pd.to_datetime(trg_events['event_ts'], utc = False).dt.tz_convert('UTC')

In [40]:
trg_events

,VEHICLE_ID,event_ts,event
0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 02:23:54+00:00,ignitionOn
1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 02:24:28+00:00,ignitionOff
2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 20:09:00+00:00,ignitionOn
3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 20:12:34+00:00,ignitionOff
4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-01 21:05:03+00:00,ignitionOn
...,...,...,...
9930,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-30 23:02:03+00:00,ignitionOff
9931,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-30 23:12:35+00:00,ignitionOn
9932,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-30 23:12:35+00:00,ignitionOn
9933,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-30 23:21:53+00:00,ignitionOff


In [41]:
ign_off_data['event_ts'] = pd.to_datetime(ign_off_data['event_ts'], utc = True, errors = 'coerce')

invalid = ign_off_data['event_ts'].isna()
if invalid.any():
    print(f"[WARN] {invalid.sum()} invalid SYN timestamps dropped")
ign_off_data = ign_off_data[['VEHICLE_ID', 'event_ts', 'event']].dropna(subset = ['event_ts'])

In [42]:
ign_off_data

,VEHICLE_ID,event_ts,event
402,19c4f06a-d5c7-4e51-ae2c-56c5367c0d50,2021-12-19 07:31:53.009000+00:00,ignitionOff
401,19c4f06a-d5c7-4e51-ae2c-56c5367c0d50,2021-12-19 18:33:46.004000+00:00,ignitionOff
393,19c4f06a-d5c7-4e51-ae2c-56c5367c0d50,2021-12-20 17:45:08.022000+00:00,ignitionOff
380,19c4f06a-d5c7-4e51-ae2c-56c5367c0d50,2021-12-21 22:52:04.009000+00:00,ignitionOff
363,19c4f06a-d5c7-4e51-ae2c-56c5367c0d50,2021-12-22 17:35:47.008000+00:00,ignitionOff
...,...,...,...
104,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-20 22:18:24.334000+00:00,ignitionOff
97,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-22 05:18:25.213000+00:00,ignitionOff
80,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-24 00:36:40.015000+00:00,ignitionOff
24,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2022-01-29 02:14:24.006000+00:00,ignitionOff


In [43]:
all_events = pd.concat([
    tlm_events.assign(source = 'TLM'),
    trg_events.assign(source = 'TRG'),
    ign_off_data.assign(source = 'SYN')], ignore_index = True)

In [44]:
all_events = all_events.sort_values(['VEHICLE_ID', 'event_ts', 'source'])

In [45]:
all_events['event_ts'] = pd.to_datetime(all_events['event_ts'], utc = True, errors = 'coerce')
all_events['ts_bucket'] = all_events['event_ts'].dt.floor('5S')
grouped = all_events.groupby(['VEHICLE_ID', 'ts_bucket', 'event'])
deduped = grouped.agg(event_ts = ('event_ts', 'min'), sources = ('source', lambda x: list(set(x)))).reset_index()

C:\Users\theor\AppData\Local\Temp\ipykernel_16088\749485728.py:2: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  all_events['ts_bucket'] = all_events['event_ts'].dt.floor('5S')


In [46]:
priority = {'TLM': 0, 'TRG': 1, 'SYN': 2}
deduped['best_source'] = deduped['sources'].apply(lambda x: min(x, key = lambda s: priority.get(s, 3)))
deduped = deduped.sort_values('best_source').drop_duplicates(['VEHICLE_ID', 'ts_bucket', 'event'])
deduped = deduped[['VEHICLE_ID', 'event_ts', 'event']].sort_values(['VEHICLE_ID', 'event_ts'])

In [47]:
deduped['prev_event'] = deduped.groupby('VEHICLE_ID')['event'].shift(1)
mask = (deduped['prev_event'] != deduped['event']) | deduped['prev_event'].isna()
final_ign = deduped[mask].copy()
final_ign = final_ign[['VEHICLE_ID', 'event', 'event_ts']].rename(columns = {'VEHICLE_ID': 'vehicle_id'})
final_ign.to_csv('IgnitionEvents.csv', index = None)

In [48]:
trg_events = trg_data[trg_data['NAME'] == 'EV_CHARGE_STATE']
trg_events['event'] = trg_events['VAL']
trg_events = trg_events.merge(map_data, on = ['PNID'])
trg_events = trg_events[['ID', 'CTS', 'event']].rename(columns = {'ID': 'vehicle_id', 'CTS': 'event_ts'})
trg_events['event_ts'] = pd.to_datetime(trg_events['event_ts'], errors = 'coerce')
trg_events['event_ts'] = trg_events['event_ts'].dt.tz_convert('UTC')

C:\Users\theor\AppData\Local\Temp\ipykernel_16088\4191913460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg_events['event'] = trg_events['VAL']


In [49]:
trg_events = trg_events.sort_values(['vehicle_id', 'event_ts'])
trg_events['prev_event'] = trg_events.groupby('vehicle_id')['event'].shift(1)
trg_events = trg_events[(trg_events['event'] != trg_events['prev_event']) | trg_events['prev_event'].isna()]
trg_events = trg_events[['vehicle_id', 'event', 'event_ts']].reset_index(drop = True)

In [50]:
trg_events.to_csv('ChargingStatusEvents.csv', index = None)

In [51]:
status = trg_events.copy()

In [65]:
charge_state = trg_data[trg_data['NAME'] == 'CHARGE_STATE']
charge_state = charge_state.merge(map_data, on = 'PNID')
charge_state = charge_state.rename(columns = {'ID': 'vehicle_id', 'CTS': 'event_ts'})
charge_state['battery_level'] = pd.to_numeric(charge_state['VAL'], errors = 'coerce')
charge_state = charge_state[['vehicle_id', 'event_ts', 'battery_level']]
charge_state = charge_state.sort_values(['vehicle_id', 'event_ts'])

In [81]:
status['event_ts'] = pd.to_datetime(status['event_ts'], utc = True)
status = status.sort_values(['vehicle_id', 'event_ts'])

In [75]:
active = status[status['event'] == 'Active']
end_events = status[status['event'].isin(['Complete', 'Abort'])]

In [76]:
def get_level_at_ts(df, vehicle_id, target_ts, window_min = 2):
    window = pd.Timedelta(minutes=window_min)
    veh_data = df[df['vehicle_id'] == vehicle_id]
    in_window = veh_data[
        (veh_data['event_ts'] >= target_ts - window) &
        (veh_data['event_ts'] <= target_ts + window)
    ]
    if in_window.empty:
        return None
    closest = in_window.iloc[((in_window['event_ts'] - target_ts).abs()).argsort()[:1]]
    return closest['battery_level'].iloc[0]

In [93]:
def session_has_engine_on_vectorized(vehicle_id, start_ts, end_ts, ign_on, ign_off):
    veh_on = ign_on[ign_on['vehicle_id'] == vehicle_id]
    veh_off = ign_off[ign_off['vehicle_id'] == vehicle_id]

    if veh_on.empty:
        return False
    intervals = []
    off_idx = 0
    off_times = veh_off['ign_off_ts'].values

    for on_ts in veh_on['ign_on_ts'].values:
        # find first off >= on
        while off_idx < len(off_times) and off_times[off_idx] < on_ts:
            off_idx += 1
        off_ts = off_times[off_idx] if off_idx < len(off_times) else pd.Timestamp('2100-01-01', tz='UTC')
        intervals.append(pd.Interval(on_ts, off_ts, closed='both'))

    if not intervals:
        return False

    # build IntervalIndex
    interval_idx = pd.IntervalIndex(intervals)

    # vectorized overlap check
    query_interval = pd.Interval(start_ts, end_ts, closed='both')
    return interval_idx.overlaps(query_interval)

In [85]:
def session_has_engine_on(vehicle_id, start_ts, end_ts, ign_on, ign_off):
    veh_on = ign_on[ign_on['vehicle_id'] == vehicle_id]
    veh_off = ign_off[ign_off['vehicle_id'] == vehicle_id]
    for _, on_row in veh_on.iterrows():
        on_ts = on_row['ign_on_ts']
        off_candidates = veh_off[veh_off['ign_off_ts'] >= on_ts]
        off_ts = off_candidates.iloc[0]['ign_off_ts'] if not off_candidates.empty else pd.Timestamp('2100-01-01', tz='UTC')
        if not (end_ts < on_ts or start_ts > off_ts):
            return True
    return False

In [87]:
ign_on = final_ign[final_ign['event'] == 'ignitionOn'][['vehicle_id', 'event_ts']].rename(columns = {'event_ts': 'ign_on_ts'})
ign_off = final_ign[final_ign['event'] == 'ignitionOff'][['vehicle_id', 'event_ts']].rename(columns = {'event_ts': 'ign_off_ts'})

In [97]:
ign_off = ign_off.dropna(subset = ['ign_off_ts'])
ign_on = ign_on.dropna(subset = ['ign_on_ts'])

In [91]:
threshold_on = 5
threshold_off = 10

In [108]:
ignition_intervals = {}
for veh in ign_on['vehicle_id'].unique():
    on_times = ign_on[ign_on['vehicle_id'] == veh]['ign_on_ts'].sort_values().values
    off_times = ign_off[ign_off['vehicle_id'] == veh]['ign_off_ts'].sort_values().values

    intervals = []
    off_idx = 0
    for on_ts in on_times:
        while off_idx < len(off_times) and off_times[off_idx] < on_ts:
            off_idx += 1
        if off_idx >= len(off_times):
            break
        off_ts = off_times[off_idx]
        on_ts = pd.Timestamp(on_ts, tz = 'UTC')
        off_ts = pd.Timestamp(off_ts, tz = 'UTC')
        # Now off_ts is guaranteed to be valid pd.Timestamp
        # try:
        intervals.append(pd.Interval(on_ts, off_ts, closed='both'))
        # except:
        #     print(type(on_ts))
        #     print(type(off_ts))

    if intervals:
        ignition_intervals[veh] = pd.IntervalIndex(intervals)

In [110]:
events = []
for _, start_row in active.iterrows():
    veh = start_row['vehicle_id']
    start_ts = start_row['event_ts']

    # find next end event
    candidates = end_events[
        (end_events['vehicle_id'] == veh) &
        (end_events['event_ts'] > start_ts)
    ]
    if candidates.empty:
        continue
    end_row = candidates.iloc[0]
    end_ts = end_row['event_ts']

    # duration_min = (end_ts - start_ts).total_seconds() / 60
    # if duration_min < 5:
    #     continue

    # delta from charge_state
    start_level = get_level_at_ts(charge_state, veh, start_ts)
    end_level = get_level_at_ts(charge_state, veh, end_ts)

    if start_level is None:
        start_level = 0
    if end_level is None:
        end_level = 100

    delta = max(0, end_level - start_level)

    # check ignition
    # engine_on = session_has_engine_on_vectorized(veh, start_ts, end_ts, ign_on, ign_off)
    
    engine_on = (
        veh in ignition_intervals and
        ignition_intervals[veh].overlaps(pd.Interval(start_ts, end_ts, closed='both')).any())
    if engine_on:
        ignition_state = 'ON'
    else:
        ignition_state = 'OFF'

    # threshold
    threshold = threshold_on if ignition_state == 'ON' else threshold_off
    if delta < threshold:
        continue

    events.append({
        'vehicle_id': veh,
        'start_ts': start_ts,
        'end_ts': end_ts,
        'delta_battery_pct': round(delta, 2),
        'ignition_state': ignition_state
    })

In [115]:
result = pd.DataFrame(events)
if not result.empty:
    result['start_ts'] = result['start_ts'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
    result['end_ts'] = result['end_ts'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'

result = result.sort_values(['vehicle_id', 'start_ts']).reset_index(drop = True)
result.drop_duplicates().to_csv('ChargingEvents.csv', index = None)